In [ ]:
#Gene Set Enrichment Analysis using Fisher Exact Test

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
import pickle
import seaborn as sns
from scipy.stats import fisher_exact
import gseapy as gp

In [ ]:
#Read in the required data

In [ ]:
datExpr = pd.read_csv('/data/recode/mic_datExpr_pseudobulk.csv', index_col = 0)

In [ ]:
with open('/data/recode/separated_communities.pkl', 'rb') as file:
        separated_communities = pickle.load(file)

In [ ]:
# Convert each module in separated_communities to a set
modules = {f'Module_{i+1}': set(community) for i, community in enumerate(separated_communities)}

In [ ]:
modules

In [ ]:
# Step 1: Load the GMT file
def read_gmt_file(file_path):
    gene_sets = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            gene_set_name = parts[0]
            genes = parts[2:]
            gene_sets[gene_set_name] = set(genes)
    return gene_sets

In [ ]:
gmt_file = 'h.all.v2023.2.Hs.symbols.gmt'
gene_sets = read_gmt_file(gmt_file)

In [ ]:
gene_sets

In [ ]:
#Calculate Fisher's Exact Test for Enrichment

# Container for results
results = []

# Iterate over each module
for module_name, module_genes in modules.items():
    module_genes_in_expr = module_genes.intersection(datExpr.columns)
    
    # Check if module_genes_in_expr is empty to avoid issues
    if len(module_genes_in_expr) == 0:
        continue

    # Iterate over each gene set in the GMT file
    for gene_set_name, gene_set_genes in gene_sets.items():
        overlap_genes = module_genes_in_expr.intersection(gene_set_genes)
        non_overlap_module_genes = module_genes_in_expr.difference(gene_set_genes)
        non_overlap_gene_set_genes = gene_set_genes.difference(module_genes_in_expr)

        # Calculate the contingency table
        a = len(overlap_genes)
        b = len(non_overlap_module_genes)
        c = len(non_overlap_gene_set_genes)
        d = len(set(datExpr.columns)) - (a + b + c)  # Adjust d to only consider genes in datExpr
        
        # Ensure all values in the contingency table are non-negative
        if a < 0 or b < 0 or c < 0 or d < 0:
            continue

        # Perform Fisher's exact test
        odds_ratio, p_value = fisher_exact([[a, b], [c, d]], alternative='two-sided')
        
        results.append({
            'Module': module_name,
            'Gene Set': gene_set_name,
            'Overlap Count': a,
            'Odds Ratio': odds_ratio,
            'P-value': p_value
        })


In [ ]:
# Convert results to DataFrame for further analysis
results_df = pd.DataFrame(results)

# Adjust p-values for multiple testing
results_df['Adjusted P-value'] = multipletests(results_df['P-value'], method='fdr_bh')[1]

# Display results
print(results_df)

In [ ]:
#Let's focus on one module for the purpose of the enrichment analysis:
# Filter for Module 1 and sort by overlap count
module_1_results = results_df[results_df['Module'] == 'Module_1']
top_10_enriched_gene_sets = module_1_results.sort_values(by='Overlap Count', ascending=False).head(10)

print(top_10_enriched_gene_sets)

In [ ]:
#Extract data for plotting (top 10 pathways with highest overlap count)
top_10_enriched_gene_sets
y_labels = top_10_enriched_gene_sets['Gene Set']
x = top_10_enriched_gene_sets['Odds Ratio']
sizes = top_10_enriched_gene_sets['Overlap Count'] * 20
p_values = top_10_enriched_gene_sets['P-value']
adjusted_p_values = top_10_enriched_gene_sets['Adjusted P-value']

In [ ]:
# Plot the scatter plot for the top 10 pathways
plt.figure(figsize=(12, 8))

# Main scatter plot
sc = plt.scatter(x, range(10), s=sizes, c=-adjusted_p_values, cmap='viridis', alpha=0.7)

# Set labels and title
plt.yticks(range(10), y_labels)  # Set y-axis labels to pathway names
plt.xlabel('Odds Ratio')
plt.ylabel('Pathway')
plt.title('Top 10 Enriched Gene Sets')

# Add a legend for dot sizes (overlap count) outside the plot
handles, labels = sc.legend_elements(prop="sizes", alpha=0.6)
legend_labels = [f'Overlap Count: {int(float(label.split("{")[1].split("}")[0]) // 20)}' for label in labels]
legend = plt.legend(handles, legend_labels, loc="center left", bbox_to_anchor=(1.3, 0.5), title="Overlap", frameon=False)

# Add colorbar
cbar = plt.colorbar(sc, shrink=0.2)  # Shrink colorbar size
cbar.ax.xaxis.set_label_position('top')
cbar.ax.set_title('Adj.pval', loc='left') 

# Adjust layout to prevent overlap between legends
plt.subplots_adjust(right=0.75)  # Adjust the right margin to make space for the legends

# Show the plot
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
#KEGG pathway enrichment analysis using gseapy

In [ ]:
kegg = gp.get_library(name='KEGG_2016', organism='Human')

In [ ]:
kegg

In [ ]:
gene_sets

In [ ]:
# Initialize an empty dictionary to store pathway name: gene names pairs
pathway_dict = {}

# Iterate through the pathway_gene_dict and populate the pathway_dict
for pathway_name, gene_list in kegg.items():
    # Convert gene list to a set to remove duplicates
    gene_set = set(gene_list)
    # Assign gene set to pathway name in pathway_dict
    pathway_dict[pathway_name] = gene_set

# Display the resulting dictionary
print(pathway_dict)

In [ ]:
# Container for results
results = []

# Iterate over each module
for module_name, module_genes in modules.items():
    module_genes_in_expr = module_genes.intersection(datExpr.columns)
    
    # Check if module_genes_in_expr is empty to avoid issues
    if len(module_genes_in_expr) == 0:
        continue

    # Iterate over each gene set in the GMT file
    for gene_set_name, gene_set_genes in pathway_dict.items():
        overlap_genes = module_genes_in_expr.intersection(gene_set_genes)
        non_overlap_module_genes = module_genes_in_expr.difference(gene_set_genes)
        non_overlap_gene_set_genes = gene_set_genes.difference(module_genes_in_expr)

        # Calculate the contingency table
        a = len(overlap_genes)
        b = len(non_overlap_module_genes)
        c = len(non_overlap_gene_set_genes)
        d = len(set(datExpr.columns)) - (a + b + c)  # Adjust d to only consider genes in datExpr
        
        # Ensure all values in the contingency table are non-negative
        if a < 0 or b < 0 or c < 0 or d < 0:
            continue

         # Perform Fisher's exact test only if all values are non-negative
        contingency_table = [[a, b], [c, d]]
        odds_ratio = None
        if b * c != 0:
            odds_ratio = (a * d) / (b * c)
        
        results.append({
            'Module': module_name,
            'Gene Set': gene_set_name,
            'Overlap Count': a,
            'Odds Ratio': odds_ratio,
            'P-value': p_value
        })

In [ ]:
# Convert results to DataFrame for further analysis
results_df = pd.DataFrame(results)

# Adjust p-values for multiple testing
results_df['Adjusted P-value'] = multipletests(results_df['P-value'], method='fdr_bh')[1]

# Display results
print(results_df)

In [ ]:
# Filter for Module 1 and sort by overlap count
module_1_results = results_df[results_df['Module'] == 'Module_1']
top_10_enriched_gene_sets = module_1_results.sort_values(by='Overlap Count', ascending=False).head(10)

print(top_10_enriched_gene_sets)

In [ ]:
# Extract data for plotting (top 10 pathways with highest overlap count)
top_10_enriched_gene_sets
y_labels = top_10_enriched_gene_sets['Gene Set']
x = top_10_enriched_gene_sets['Odds Ratio']
sizes = top_10_enriched_gene_sets['Overlap Count'] * 20
p_values = top_10_enriched_gene_sets['P-value']
adjusted_p_values = top_10_enriched_gene_sets['Adjusted P-value']

In [ ]:
# Plot the scatter plot for the top 10 pathways
plt.figure(figsize=(12, 8))

# Main scatter plot
sc = plt.scatter(x, range(10), s=sizes, c=-adjusted_p_values, cmap='viridis', alpha=0.7)

# Set labels and title
plt.yticks(range(10), y_labels)  # Set y-axis labels to pathway names
plt.xlabel('Odds Ratio')
plt.ylabel('Pathway')
plt.title('Top 10 Enriched Pathways')

# Add a legend for dot sizes (overlap count) outside the plot
handles, labels = sc.legend_elements(prop="sizes", alpha=0.6)
legend_labels = [f'Overlap Count: {int(float(label.split("{")[1].split("}")[0]) // 20)}' for label in labels]
legend = plt.legend(handles, legend_labels, loc="center left", bbox_to_anchor=(1.3, 0.5), title="Overlap", frameon=False)

# Add colorbar
cbar = plt.colorbar(sc, shrink=0.2)  # Shrink colorbar size
cbar.ax.xaxis.set_label_position('top')
cbar.ax.set_title('Adj.pval', loc='left') 

# Adjust layout to prevent overlap between legends
plt.subplots_adjust(right=0.75)  # Adjust the right margin to make space for the legends

# Show the plot
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
#Exercise Questions:
#1.What is gene set enrichment analysis?
#2.What is Fisher Exact Test?
#3.How does GSEA differ from use of Fisher Exact Test?

In [ ]:
#Answers
#1.GSEA is a computational method used to determine whether predefined sets of genes (gene sets) exhibit statistically significant differences in their expression between two or more biological states or conditions.
#2.The Fisher Exact Test is a statistical significance test used to determine whether there are nonrandom associations between two categorical variables in a contingency table. 
#3.GSEA focuses on analysing the collective behavior of genes within gene sets to uncover biologically relevant pathways or functional modules.
#Whereas Fisher Exact Test assesses the significance of associations between individual genes or genomic features and phenotypic categories.
#GSEA assesses the enrichment of gene sets by comparing the distribution of genes within each set to the overall distribution of genes ranked based on a statistical metric (e.g., differential expression score). It uses a running sum statistic or similar approach to calculate an enrichment score for each gene set and assesses statistical significance based on permutation testing or other methods.
#Whereas Fisher Exact Test alculates the probability of obtaining the observed distribution of frequencies in a contingency table under the null hypothesis of independence between two categorical variables. It uses the hypergeometric distribution to compute the probability of obtaining as extreme or more extreme frequencies than the observed frequencies.